In [4]:
import cvxopt
import cvxpy
import numpy as np
from qsweepy import *
import json
#%matplotlib qt5
import matplotlib.pyplot as plt

In [3]:
%matplotlib tk

In [5]:
measurement_results = {}
measurement_results_np = np.zeros((6*6*6*6*4))
projection_order = ['+Z', '+X', '+Y', '-Z', '-X', '-Y']
measurement_id = 0
initials = np.zeros((6*6, 16), dtype=np.complex)
measurement_operators = np.zeros((16, 16*16*4), dtype=np.complex)
with open(r'C:\Users\cryopxi\Documents\CNOT\CX-experiment-20181115.txt', 'r') as f:
    for p3_id, p3 in enumerate(projection_order):
        for p4_id, p4 in enumerate(projection_order):
            for p1_id, p1 in enumerate(projection_order):
                for p2_id, p2 in enumerate(projection_order):
                    initial_id = 
                    for ri,rj in itertools.product([0,1], [0,1]):
                        projection_id = p1_id*4*6+p2_id*4+ri*2+rj
                        
                        _matrix = np.zeros((4,4), dtype=complex)
                        _matrix[ri*2+rj, ri*2+rj] = 1
                        readout_result = str(rj)+str(ri)
                        q1_unitary = cube_faces_unitaries[p3]
                        q2_unitary = cube_faces_unitaries[p4]
                        unitary = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary, q2_unitary), (4,4)), np.complex)
                        
                        q1_unitary_pre = cube_faces_unitaries[p1]
                        q2_unitary_pre = cube_faces_unitaries[p2]
                        unitary_pre = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary_pre, q2_unitary_pre), (4,4)), np.complex)
                    
                        initial = np.dot(unitary_pre, np.dot(initial_state, np.conj(unitary_pre).T)).ravel()
                        projection = np.dot(np.conj(unitary.T), np.dot(_matrix, unitary)).T.ravel()
    
                        initials[]
    
                        # read data from file
                        projection_name = f.readline()
                        if len(projection_name)<2:
                            projection_name = f.readline()
                        uncorrected = f.readline()
                        corrected = f.readline()
    
                        corrected_json = json.loads(corrected[10:].replace('\'', '"'))
                        measurement_results[p1+p2+p3+p4] = corrected_json
                        measurement_results_np[measurement_id] = measurement_results[p1+p2+p3+p4][readout_result]
                    
                        measurement_id += 1

In [ ]:
chi_dimension =16
reconstruction_basis = {}
proj_seq = {}
import itertools

sigma_x = np.asarray([[0,1],  [1,0]])
sigma_y = np.asarray([[0,-1j],[1j,0]])
sigma_z = np.asarray([[1,0],  [0,-1]])
sigma_i = np.asarray([[1,0],  [0,1]])

cube_faces = {'+Z':(0.0, 0.0), 
              '+X':(np.pi/2., -np.pi/2.,),
              '+Y':(np.pi/2., 0.0),
              '-Z':(np.pi, 0.0),
              '-X':(np.pi/2., np.pi/2.),
              '-Y':(np.pi/2., np.pi)}

#cube_faces = {'+Z':(0.0, 0.0), '-Z':(np.pi, 0.0)}

cube_faces_unitaries = {k:    np.cos(v[0]/2)*sigma_i-\
                           1j*np.sin(v[0]/2)*np.cos(v[1])*sigma_x-\
                           1j*np.sin(v[0]/2)*np.sin(v[1])*sigma_y for k,v in cube_faces.items()}

initial_state = np.zeros((4,4))
initial_state[0,0] = 1.

measurement_id = 0
for q1_rotation_name, q1_rotation in cube_faces.items():
    for q2_rotation_name, q2_rotation in cube_faces.items():
        for q1_pre_name, q1_pre in cube_faces.items():
            for q2_pre_name, q2_pre in cube_faces.items():
                pulses_pre = get_rotation('1', *q1_pre)+get_rotation('3', *q2_pre)
                pulses = get_rotation('1', *q1_rotation)+get_rotation('3', *q2_rotation)+ro_seq
                operators = {}

                for ri,rj in itertools.product([0,1], [0,1]):
                    _matrix = np.zeros((4,4), dtype=complex)
                    _matrix[ri*2+rj, ri*2+rj] = 1
                    readout_result = str(rj)+str(ri)
                    q1_unitary = cube_faces_unitaries[q1_rotation_name]
                    q2_unitary = cube_faces_unitaries[q2_rotation_name]
                    unitary = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary, q2_unitary), (4,4)), np.complex)
                    
                    q1_unitary_pre = cube_faces_unitaries[q1_pre_name]
                    q2_unitary_pre = cube_faces_unitaries[q2_pre_name]
                    unitary_pre = np.asarray(np.reshape(np.einsum('ij,kl->ikjl', q1_unitary_pre, q2_unitary_pre), (4,4)), np.complex)
                    
                    initial = np.dot(unitary_pre, np.dot(initial_state, np.conj(unitary_pre).T)).ravel()
                    projection = np.dot(np.conj(unitary.T), np.dot(_matrix, unitary)).T.ravel()
                    
                    superoperator_projection = np.einsum('i,j->ij', projection, initial)
                    
                    operators[readout_result] = superoperator_projection
                    h[0,1+measurement_id] = measurement_results[p1+p2+p3+p4][rj+ri]
                    h[1+measurement_id,0] = measurement_results[p1+p2+p3+p4][rj+ri]
                    #h superoperation_projection
                    measurement_id +=1
                #proj_seq[q1_pre_name+q2_pre_name+q1_rotation_name+q2_rotation_name] = \
                #    {'pre_pulses': pulses_pre, 'pulses':pulses, 'operators':operators}

single_qubit_chi_basis = np.arange(16)           
for i,j in itertools.product(single_qubit_chi_basis, single_qubit_chi_basis):
    _matrix = np.zeros((16,16))
    _matrix[i,j]=1.
    reconstruction_basis[str(i)+','+str(j)] = {'operator': _matrix}

#two_qubit_process_tomography = multiqubit_tomography.multiqubit_tomography(multiqubit_state_probabilities,
#                                                                          pg, 
#                                                                          proj_seq,
#                                                                          reconstruction_basis)
#two_qubit_process_tomography.readout_names = readout_result_order
#two_qubit_process_tomography.confusion_matrix = confusion_matrix_elements

In [4]:
len(measurement_results)

1296

In [7]:

constraint_matrix_size = 1+len(measurement_results)+chi_dimension*2

h = cvxopt.matrix(np.zeros((constraint_matrix_size,constraint_matrix_size)))
h[1:1+len(measurement_results), 1:1+len(measurement_results)]=np.identity(len(measurement_results))
h = [h]
h

[<1329x1329 matrix, tc='d'>]

In [19]:
G = np.zeros((chi_dimension*chi_dimension+1, constraint_matrix_size*constraint_matrix_size))
#G = [cvxopt.matrix(G)]
G[0,0]=1.
G[


In [20]:
G

[<257x1723969 matrix, tc='d'>]

In [ ]:
#generate two (arbitrary) positive hermitian operators
P = cvx.matrix([ [1-1j , 2+2j  , 1    ],
                 [3j   , -2j   , -1-1j],
                 [1+2j, -0.5+1j, 1.5  ]
                ])
P = P * P.H

Q = cvx.matrix([ [-1-2j , 2j   , 1.5   ],
                 [1+2j  ,-2j   , 2.-3j ],
                 [1+2j  ,-1+1j , 1+4j  ]
                ])
Q = Q * Q.H

n=P.size[0]
P = pic.new_param('P',P)
Q = pic.new_param('Q',Q)

#create the problem in picos

F = pic.Problem()
Z = F.add_variable('Z',(n,n),'complex')

F.set_objective('max','I'|0.5*(Z+Z.H))       #('I' | Z.real) works as well
F.add_constraint(((P & Z) // (Z.H & Q))>>0 )

print(F)

F.solve(verbose = 0, solver = 'cvxopt')

print('fidelity: F(P,Q) = {0:.4f}'.format(F.obj_value()))

print('optimal matrix Z:')
print(Z)

In [12]:
x = cvxpy.Variable((chi_dimension,chi_dimension), hermitian=True)
x

Variable((16, 16), hermitian=True)

In [7]:
cvxpy.Variable?

In [ ]:
# Entropy maximization.
x = cvx.Variable(shape=(chi_dimension,chi_dimension), PSD=True, complex=True, hermitian=True)
obj = cvx.Maximize(cvx.sum(cvx.entr(x)))
constraints = [A*x == b,
               F*x <= g ]
prob = cvx.Problem(obj, constraints)
prob.solve(solver=cvx.CVXOPT, verbose=True)

# Print result.
print "\nThe optimal value is:", prob.value
print '\nThe optimal solution is:'
print x.value